# Demonstration analysis
This Jupyter notebook demonstrates the functionality of `phase.py` in an example phase retrieval routine. The data used here is a sample of real measurements.

In [ ]:
%pylab inline
from mpl_toolkits.axes_grid1 import ImageGrid
import scipy.optimize as op
from phase import *

def imshow_grid(imgs, frac=1., clims=None):
    """A wrapper to conveniently display zoomed images in a grid."""
    n = len(imgs)
    fig = figure(figsize=(4*n, 3))
    img_grid = ImageGrid(fig, 111, nrows_ncols=(1, n), axes_pad=.75, cbar_mode='each', cbar_pad='5%')
    for i, (ax, cbar, img) in enumerate(zip(img_grid, img_grid.cbar_axes, imgs)):
        N = img.shape[0] / 2.
        M = img.shape[1] / 2.
        im = ax.imshow(img, extent=(-M, M, N, -N), interpolation='nearest')
        ax.set_xlim(-frac*M, frac*M)
        ax.set_ylim(-frac*N, frac*N)
        if (clims is not None) and (clims[i] is not None):
            im.set_clim(clims[i])
        cbar.colorbar(im)

## Estimating the focal plane position
Defocused pinhole images are loaded from a sample data file. The function `analyze_peaks` provides normalized peak intensities, which are then used to fit a $\mathrm{sinc}^2$ function and estimate the focal plane location. Images in the file `focus.tif` were taken with a 1 mm spacing.

In [ ]:
stack = tifffile.imread('demo_data.tif')
Z = arange(len(stack))   # 1 mm spacing
p, v = analyze_peaks(stack, 32, 16, index=Z)

In [ ]:
def sinc2(x, A, a, x0, B):
    """sinc^2 function for fitting to through-focus intensities."""
    return A * sinc(a * (x - x0)) ** 2 + B

In [ ]:
pfit, cfit = op.curve_fit(sinc2, Z, p['norm amp'].astype(float64), p0=(2e-2, .5e-3, (Z[0] + Z[-1]) / 2., 0.))
plt.plot(Z - pfit[2], p['norm amp'], '.')
plt.plot(Z - pfit[2], sinc2(Z, *pfit), '-')
for pname, pval, sval in zip(('A', 'a', 'x0', 'B'), pfit, np.sqrt(np.diag(cfit))):
    print(pname, '=', pval, '+/-', sval)

## Preparing images
Images are cropped to the specified size, each frame is centered at the maximum. Then, backgrounds are subtracted and images are normalized. A square root of each of them is taken to get complex amplitudes.

In [ ]:
### Some physical quantities and settings are defined here.
dx = 6.8         # Pixel size [um]
z0 = 473000./dx  # Distance to the exit pupil [px]
wl = .766/dx     # Wavelength [px]
N = 256          # Image width [px]
n = 4            # Number of images to be used in a single optimization
# Preparing FFTs.
#tr = Transforms(N)  
#tr = Transforms_FFTW(N)  
tr = Transforms_CUDA(N)  

In [ ]:
# Transforming distances to values in pixels 
# relative to the focal plane position.
Z = (Z - pfit[2]) * 1e+3 / dx     
F = zeros((len(stack), N, N), dtype=float64)
for i, page in enumerate(stack):
    img = crop(page, p.loc[i,'x0'], p.loc[i,'y0'], s=N).astype(complex128)
    img = sqrt((img - p.loc[i,'bg']) / p.loc[i,'tot power'])
    # Conserving the zero-mean noise to avoid bias. 
    # Negative values in amplitudes are allowed.
    F[i] = img.real - img.imag

Picking 4 images for optimization, including the master plane determined by index `j`.

In [ ]:
idx = array([0, 6, 11, 16])
imshow_grid(F[idx], frac=.25)

## Phase retrieval
Phase in the master plane is estimated via non-linear optimization. The optimization routine starts with random uniform phase. The L-BFGS method is used to minimize the error metric.

In [ ]:
zj = Z[idx[0]]
zk = Z[idx[1:]]
Fj = F[idx[0]]
Fk = F[idx[1:]]
#errf = Errf(zj, zk, Fj, Fk, wl)
#errf = Errf_FFTW(zj, zk, Fj, Fk, wl)
errf = Errf_CUDA(zj, zk, Fj, Fk, wl)

In [ ]:
ph0 = 2. * pi * (rand(N**2) - .5)
niter = [0]
nout = 50
def print_info(ph):
    if not niter[0]%nout:
        E, dE = errf(ph)
        print('%4d\t%10.4e  %10.4e' % (niter[0], E, abs(dE).max()))
    niter[0] += 1
              
opt_ph = op.minimize(errf, x0=ph0, method='L-BFGS-B', jac=True, options={'maxiter':500}, callback=print_info)

Phase in the master plane is obtained directly from the optimization algorithm.

In [ ]:
ph = opt_ph.x.reshape((N,N))
ph_jac = opt_ph.jac.reshape((N,N))
# This wraps the phase, making it easier to display on a graph.
P = np.exp(1.j*ph)  
imshow_grid((Fj, angle(P)/2./pi, ph_jac), frac=.25)

**Left:** master plane amplitude. **Center:** reconstructed phase in wavelengths. **Right:** gradient after the last iteration.

ASP to the focal plane and backward Fraunhofer diffraction to the exit pupil yield the exit pupil field, whose complex phase corresponds to the wavefront aberration.

In [ ]:
G0 = tr.fraun(tr.asp(Fj * P, -zj, wl), -z0, wl)
# This roughly subtracts the piston, better for displaying.
G0 *= exp(-1.j * angle(sum(G0)))
imshow_grid((abs(G0), angle(G0) / 2. / pi), frac=.25, clims=(None, (-.1,.15)))

**Left:** pupil field amplitude. **Right:** phase in wavelengths, i.e. the wavefront error.

## Pupil radius and location

A top-hat function representing the circular opening of the exit pupil is fitted. This allows to extract the pupil radius and a spatial shift due to phase tilts in the focal plane.

In [ ]:
def fit_circ(par, img):
    A, B, dx, dy, a = par
    circ = circle(N/2+dx, N/2+dy, a, N)*A + B
    return (circ - img).ravel()

In [ ]:
opt_circ = op.leastsq(fit_circ, (abs(G0).max(), 0., 0., 0., 30), args=(abs(G0),), full_output=True)
res_circ = opt_circ[2]['fvec'].reshape((N,N))  # residuals
imshow_grid((abs(G0), abs(G0)+res_circ, res_circ), frac=.25)
for pname, p in zip(('A', 'B', 'dx', 'dy', 'a'), opt_circ[0]):
    print(pname, '=', p)
a = opt_circ[0][-1]

**Left:** exit pupil amplitude. **Center:** the fitted top-hat. **Right:** fit residuals.

The fitted pupil location is used to correct phase tilts in the focal plane. Reconstruction is then repeated, resulting in the pupil being centered. This may not be necessary in this case, but in two-image reconstructions pupil shifts may be significantly larger.

In [ ]:
# This is a Fourier shift.
S = ifftshift(exp(2.j * pi * (tr.x*opt_circ[0][2] + tr.y*opt_circ[0][3]) / N))
G0 = tr.fraun(tr.asp(Fj * P, -zj, wl) * S, -z0, wl)
G0 *= np.exp(-1.j * angle(sum(G0)))
imshow_grid((abs(G0), angle(G0) / 2. / pi), frac=.25, clims=(None, (-.1,.15)))

## Extracting Zernike polynomial expansion coefficients
The RMS wavefront error is calculated from the coefficients:  
$W_\mathrm{RMS} = \sqrt{\sum_{j=4}^{j_\mathrm{max}} c_j^2}$,  
starting from $c_4$, since piston and tilts do not represent true aberrations.  

This allows to approximate the Strehl ratio: $S\approx e^{-(2\pi W_\mathrm{RMS})^2}$.  
The object space Airy disk radius in um is calculated from the fitted pupil aperture radius and known magnification.

In [ ]:
jmax = 79
zern = Zernike(jmax, a, N)
W0 = angle(G0) / 2. / pi  # the original wavefront
C = zern.fit(angle(G0) / 2. / pi, jmax)
# Setting to zero outside the pupil, ignoring piston and tilts.
W = zern(np.concatenate(((0,0,0), C[3:]))) * zern.R  
G0 *= exp(-2.j * pi * zern(C[:3])) # Removing piston and tilts.
W0 = angle(G0)/ 2. / pi * zern.R

# Wavefronts set to zero outside the pupil
imshow_grid((W0, W, W - W0), frac=.25, clims=((W0.min(),W0.max()),(W0.min(),W0.max()),None)) 
figure(figsize=(15,3))
bar(arange(3, jmax) + .5, C[3:])
xlim(3.5, jmax + .5)
minorticks_on()

Wrms = sqrt(np.sum(C[3:] **2 ))
Strehl = exp(-(2. * pi * Wrms)**2)
r0 = .61 * N / a * dx / 56.20
print('Wrms =', Wrms)
print('S =', Strehl)
print('r0 =', r0)

**Left:** the original wavefront with piston and tilts removed. **Center:** Zernike fit. **Right:** fit residuals. Phase in wavelengths, set to zero outside the pupil.  
**Bottom:** expansion coefficients.